# 1.GENERACIÓN DE SECUENCIAS
En primer lugar vamos a generar una cantidad suficiente de secuencias para tener margen para filtrarlas y quedarnos con las más probables:

In [ ]:
!pip install fair-esm
!pip install tqdm biopython numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch, math, numpy as np

# Configuración ----------
DEVICE = "cuda"
TOTAL_SEQS   = 200000
BATCH_SIZE   = 30
MAX_TOKENS   = 30
TOP_K        = 650
REP_PENALTY  = 1.4


tokenizer = AutoTokenizer.from_pretrained("nferruz/ProtGPT2")
model_ppl = AutoModelForCausalLM.from_pretrained("nferruz/ProtGPT2").to(DEVICE)

# generador en GPU
protgpt2 = pipeline(
    "text-generation",
    model="nferruz/ProtGPT2",
)

# ---------- Función perplexity ----------
def calculate_perplexity(sequence: str) -> float:
    input_ids = tokenizer.encode(sequence, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        loss = model_ppl(input_ids, labels=input_ids).loss
    return math.exp(loss)

# ---------- Generar, evaluar y almacenar ----------
seqs = {}                   # {id: (seq, ppl)}
counter = 0

while counter < TOTAL_SEQS:
    current_batch = min(BATCH_SIZE, TOTAL_SEQS - counter)
    outputs = protgpt2(
        "<|endoftext|>",
        max_length=MAX_TOKENS,
        do_sample=True,
        top_k=TOP_K,
        repetition_penalty=REP_PENALTY,
        num_return_sequences=current_batch,
        eos_token_id=0
    )

    for out in outputs:
        seq_str  = out["generated_text"]
        ppl_val  = calculate_perplexity(seq_str)
        counter += 1
        seqs[f"seq{counter:05d}"] = (seq_str, ppl_val)

    print(f"Progreso: {counter}/{TOTAL_SEQS} secuencias", end="\r")

print(f"\n✓ Generadas {len(seqs)} secuencias")

# ---------- Seleccionar el 30 % de perplexity más baja ----------
perps = [v[1] for v in seqs.values()]
p30   = 500
selected = {k: v for k, v in seqs.items() if v[1] <= p30}

print(f"✓ Seleccionadas {len(selected)} secuencias (ppl ≤ {p30:.3f})")

from textwrap import wrap   # Para partir la secuencia a 60 aa

# ---------- Guardar en FASTA ----------
with open("/content/drive/MyDrive/selected_seqs2.fasta", "w") as f:
    for sid, (sequence, ppl) in selected.items():
        # 1) Quitar saltos, espacios y el token <|endoftext|>
        clean_seq = (
            sequence.replace("\n", "")
                    .replace(" ", "")
                    .replace("<|endoftext|>", "")
        )
        # 2) Cabecera FASTA
        f.write(f">{sid} ppl={ppl:.3f}\n")
        # 3) Secuencia envuelta a 60 caracteres
        for line in wrap(clean_seq, 60):
            f.write(line + "\n")

print("✓ Archivo guardado: selected_seqs.fasta")
!head -n 10 selected_seqs.fasta   # vista rápida opcional



Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Progreso: 200000/200000 secuencias
✓ Generadas 200000 secuencias
✓ Seleccionadas 154263 secuencias (ppl ≤ 500.000)
✓ Archivo guardado: selected_seqs.fasta
head: cannot open 'selected_seqs.fasta' for reading: No such file or directory


In [ ]:
#!/usr/bin/env python3
"""
Arreglo de FASTA «sucio».

Uso:
    python fix_fasta.py entrada.fa salida.fa
"""

import sys
from pathlib import Path


def fix_fasta(in_path: str | Path, out_path: str | Path) -> None:
    header = None          # Cabecera actual
    seq_parts: list[str] = []  # Fragmentos de la secuencia

    with open(in_path, encoding="utf-8") as fin, \
         open(out_path, "w", encoding="utf-8") as fout:

        for raw in fin:
            line = raw.strip()

            # 1) Saltar líneas vacías o con el marcador
            if not line or line == "<|endoftext|>":
                continue

            # 2) Nueva cabecera ➔ escribir la anterior si existe
            if line.startswith(">"):
                if header is not None:          # Hay registro anterior
                    fout.write(header + "\n")
                    fout.write("".join(seq_parts) + "\n")
                header = line                   # Guardar cabecera nueva
                seq_parts = []                  # Reiniciar acumulador
            else:
                # 3) Acumular fragmentos de secuencia (sin espacios)
                seq_parts.append(line.replace(" ", ""))

        # 4) Escribir el último registro
        if header is not None:
            fout.write(header + "\n")
            fout.write("".join(seq_parts) + "\n")


fix_fasta("/content/selected_seqs.fasta", "selected_seqs_clean.fasta")


# FILTRADO DE SECUENCIAS PLAUSIBLES
Una vez generadas las secuencias vamos a pasar a filtrarlas:
0. Inspección de homología
1. Inspección básica (Metionina terminal, sin repeticiones de un aminoácido ni de dímeros)
2. Detección de señales tansmembrana y peptidos señal
3. ESM-1v score para quedarnos con las secuencias más plausibles.

## 0. INSPECCIÓN DE HOMOLOGÍA

In [ ]:
#!wget ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/uniref/uniref90/uniref90.fasta.gz
#!wget http://github.com/bbuchfink/diamond/releases/download/v2.1.11/diamond-linux64.tar.gz
#!tar xzf diamond-linux64.tar.gz
#!gunzip uniref90.fasta.gz
#!./diamond makedb --in uniref90.fasta -d uniref
#!./diamond blastp --query passed.fasta --db swissprot.dmnd --out diamond_output.tsv --outfmt 6 --evalue 1e-5 --max-target-seqs 1 --threads 4!

diamond v2.1.11.165 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 8
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: uniref90.fasta
Opening the database file...  [0.034s]
Loading sequences...  [11.417s]
Masking sequences...  [13.26s]
Writing sequences...  [0.884s]
Hashing sequences...  [0.295s]
Loading sequences...  [11.473s]
Masking sequences...  [12.806s]
Writing sequences...  [0.888s]
Hashing sequences...  [0.296s]
Loading sequences...  [11.95s]
Masking sequences...  [12.791s]
Writing sequences...  [0.919s]
Hashing sequences...  [0.298s]
Loading sequences...  [11.831s]
Masking sequences...  [12.817s]
Writing sequences...  [1.041s]
Hashing sequences...  [0.314s]
Loading sequences...  [12.763s]
Masking sequences...  [12

## 1. Scores ESM-1v (primer filtro de probabilidad de secuencia)

In [ ]:
# merge_fasta_simple.py
from glob import glob
from pathlib import Path
n=1
with open("scores_new.csv", "w") as fout:
        for line in open("/content/scores.csv"):
            line= line.strip().split('/n')
            for line in line:
              if n==1:
                cabecera=line[0:2]

            id=line[0]
            score=line[1]
            fout.write(f"seq{seq_id_csv},{score}\n")   # ID único: seq1, seq2, …
            seq_id_csv += 1
print(f"Listo: {seq_id_csv-1} secuencias guardadas en {Path(OUTPUT_FILE).name}")


Listo: 0 secuencias guardadas en sequences_PROTGPT.fasta


In [ ]:
!pip install fair-esm
!pip install tqdm biopython numpy


import torch
import torch.nn.functional as F
import esm
from tqdm import tqdm
from Bio import SeqIO
import numpy as np


def score_sequence(model, alphabet, seq, device):
    # Filtrar caracteres no válidos para evitar KeyError
    valid = set(alphabet.tok_to_idx.keys())
    seq = ''.join([c for c in seq if c in valid])
    if len(seq) == 0:
        raise ValueError("Sequence contains no valid amino acids after filtering.")
    # Tokenizar y mover a device
    # Tokenizar y mover a device
    batch_converter = alphabet.get_batch_converter()
    _, _, toks = batch_converter([("seq", seq)])
    toks = toks.to(device)
    # Forward pass
    with torch.no_grad():
        out = model(toks, repr_layers=[], return_contacts=False)
    logits = out["logits"][0, 1:1+len(seq)]  # (L, vocab)
    log_probs = F.log_softmax(logits, dim=-1)
    token_ids = toks[0, 1:1+len(seq)]
    pos_logps = log_probs[torch.arange(len(seq)), token_ids]
    return pos_logps.mean().item()


def main():
    # --- Parámetros predefinidos ---
    input_fasta   = "/content/selected_seqs.fasta"
    model_name    = "esm1v_t33_650M_UR90S_1"
    device        = "cuda"  # "cpu" o "cuda"
    # Define un umbral fijo o un cuantíl
    threshold   = -0.13
    quantile     = 0.9
    output_fasta  = "passed.fasta"
    output_csv    = "scores.csv"
    # -------------------------------

    # Configura dispositivo
    device = torch.device(device if torch.cuda.is_available() and device.startswith("cuda") else "cpu")

    # Cargar modelo y alfabeto
    print(f"[INFO] Cargando modelo {model_name} en {device}")
    model, alphabet = esm.pretrained.load_model_and_alphabet(model_name)
    model = model.eval().to(device)

    # Leer FASTA
    records = list(SeqIO.parse(input_fasta, "fasta"))
    print(f"[INFO] {len(records)} secuencias cargadas desde {input_fasta}")

    # Calcular scores
    results = []
    for rec in tqdm(records, desc="Scoring"):
        # Limpiar secuencia: solo letras (A-Z)
        raw_seq = str(rec.seq)
        seq = ''.join(c for c in raw_seq if c.isalpha())
        if not seq:
            print(f"[WARN] Secuencia {rec.id} vacía tras limpieza, se omite.")
            continue
        score = score_sequence(model, alphabet, seq, device)
        results.append((rec.id, rec.seq, score))

    # Determinar umbral(rec.id, rec.seq, score))

    # Determinar umbral
    thr=-0.2

    # Filtrar y escribir CSV y FASTA
    passed_ids = []
    with open(output_csv, "w") as csvf:
        csvf.write("id,score,passed")
        for ident, seq, score in results:
            passed = int(score >= thr)
            csvf.write(f"{ident},{score:.6f},{passed}/n")
            if passed:
                passed_ids.append(ident)
    # Escribir FASTA resultante
    with open(output_fasta, "w") as fh:
        for rec in records:
            if rec.id in passed_ids:
                SeqIO.write(rec, fh, "fasta")

    print(f"[DONE] {len(passed_ids)}/{len(results)} secuencias pasan el filtro")

if __name__ == "__main__":
    main()

[INFO] Cargando modelo esm1v_t33_650M_UR90S_1 en cuda
[INFO] 154263 secuencias cargadas desde /content/selected_seqs.fasta


Scoring: 100%|██████████| 154263/154263 [2:14:40<00:00, 19.09it/s]


[DONE] 1523/154263 secuencias pasan el filtro


In [ ]:
input_fasta    = "/content/selected_seqs.fasta"
filter_csv     = "/content/PROTGPT_sequences_filter.csv"
ESMthreshold   = -0.25
conservate     = set()

# 1) Lee el CSV y rellena el set de secuencias a conservar
with open(filter_csv) as f:
    # Si tu CSV tiene cabecera, descomenta la siguiente línea:
    # next(f)
    n=1
    for line in f:

        if n==1:
          n=0
          continue
        ignorar,seq, score, ident = line.strip().split(",")
        seq=seq.strip('"').strip('"')
        score = float(score)
        ident = float(ident)

        # si identidad > 30% lo saltamos;
        # si score < umbral lo guardamos:
        if ident <= 35 and score > ESMthreshold:
            print(ident,score)
            conservate.add(seq)

print(f"Secuencias a conservar: {len(conservate)}")

# 2) Filtra el FASTA
output_fasta = "/content/PROTGPT_sequences_conserved.fasta"
with open(input_fasta) as fin, open(output_fasta, "w") as fout:
    write = False
    for line in fin:
        if line.startswith(">"):
            # Extrae sólo el ID (hasta el primer espacio)
            seqid = line[1:].strip().split()[0]
            write = (seqid in conservate)
        if write:
            fout.write(line)

print(f"Escrito el FASTA filtrado en: {output_fasta}")


0.0 -0.193515
0.0 -0.225741
0.0 -0.248214
0.0 -0.244754
0.0 -0.224149
0.0 -0.238012
0.0 -0.198582
0.0 -0.234612
0.0 -0.249139
0.0 -0.244382
0.0 -0.237492
0.0 -0.231801
0.0 -0.242582
0.0 -0.249099
0.0 -0.21572
0.0 -0.247029
0.0 -0.249295
0.0 -0.232227
0.0 -0.203543
0.0 -0.24027
0.0 -0.245209
0.0 -0.246452
0.0 -0.231301
0.0 -0.171452
0.0 -0.240961
0.0 -0.236204
0.0 -0.240944
0.0 -0.239726
0.0 -0.236638
0.0 -0.205149
0.0 -0.237185
0.0 -0.234918
0.0 -0.239502
0.0 -0.243957
0.0 -0.228919
0.0 -0.233601
0.0 -0.248522
0.0 -0.215414
0.0 -0.2478
0.0 -0.244294
0.0 -0.219065
0.0 -0.241627
0.0 -0.198098
0.0 -0.249696
0.0 -0.23514
0.0 -0.245279
0.0 -0.244434
0.0 -0.249866
0.0 -0.247301
0.0 -0.232127
0.0 -0.236941
0.0 -0.211141
0.0 -0.218487
0.0 -0.249227
0.0 -0.222583
0.0 -0.249751
0.0 -0.245962
0.0 -0.231967
0.0 -0.238614
0.0 -0.246025
0.0 -0.226806
0.0 -0.241144
0.0 -0.244238
0.0 -0.2443
0.0 -0.227248
0.0 -0.24799
0.0 -0.231465
0.0 -0.224741
0.0 -0.182923
0.0 -0.240025
0.0 -0.206172
0.0 -0.234485


## 2. Inspección básica y detección de señales transmembrana y péptidos señal

In [ ]:
!pip install -q git+https://github.com/BernhoferM/TMbed.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.9 MB/s eta 0:00:00


In [ ]:
#!/usr/bin/env python3
#!/usr/bin/env python3
"""
USO:
    python pipeline_tmbed.py input.fasta [out_dir]

- input.fasta : tu FASTA con todas las secuencias
- out_dir     : carpeta de salida (por defecto "./results")
La pipeline hace:
 1) Filtros de calidad (Met N-terminal, repeticiones simples y dímeros).
 2) Lanza TMbed (GPU) sobre el FASTA filtrado.
 3) Filtra las predicciones para descartar todas las proteínas que tengan
    al menos un dominio transmembrana (H/h/B/b) o signal peptide (S).
 4) Guarda un CSV con las secuencias **sin** TM ni SP y reporta totales.
 5) Genera un nuevo FASTA con las secuencias conservadas.
"""

import sys, os, csv, subprocess, tempfile

# ---------------------------------------------------------------------------
# Filtros de calidad
# ---------------------------------------------------------------------------

def pop_if_dimer_repeat(sequence, seq_id, seq_dict):
    for inicio in (0,1):
        prev = sequence[inicio:inicio+2]; count=1
        for i in range(inicio+2, len(sequence)-1, 2):
            d = sequence[i:i+2]
            if d == prev:
                count += 1
                if count >= 5:
                    seq_dict.pop(seq_id, None)
                    return True
            else:
                count = 1; prev = d
    return False


def pop_if_single_repeat(sequence, seq_id, seq_dict):
    prev = sequence[0]; k = 1
    for aa in sequence[1:]:
        if aa == prev:
            k += 1
            if k >= 4:
                seq_dict.pop(seq_id, None)
                return True
        else:
            k = 1; prev = aa
    return False

# ---------------------------------------------------------------------------
# Lectura FASTA y filtrado
# ---------------------------------------------------------------------------

def load_fasta(path):
    seqs = {}
    sid = None
    buf = []
    for L in open(path):
        if L.startswith(">"):
            if sid:
                seqs[sid] = "".join(buf).upper()
            sid = L[1:].split()[0].strip()
            buf = []
        else:
            buf.append(L.strip())
    if sid:
        seqs[sid] = "".join(buf).upper()
    return seqs


def quality_filter(seqs):
    kept = seqs.copy()
    for sid, seq in list(seqs.items()):
        if not seq.startswith("M"):
            kept.pop(sid, None); continue
        if pop_if_single_repeat(seq, sid, kept):
            continue
        pop_if_dimer_repeat(seq, sid, kept)
    return kept

# ---------------------------------------------------------------------------
# EJECUCIÓN de TMbed
# ---------------------------------------------------------------------------

def run_tmbed(fasta, out_pred):
    cmd = [
        "tmbed", "predict",
        "--fasta", fasta,
        "--predictions", out_pred,
        "--use-gpu",
        "--out-format", "0"
    ]
    subprocess.run(cmd, check=True)

# ---------------------------------------------------------------------------
# Parseo y filtrado de predicciones
# ---------------------------------------------------------------------------

def summarize_and_filter(pred_file, out_csv):
    total = discarded = kept = 0
    kept_ids = []
    with open(pred_file) as f, open(out_csv, "w", newline="") as out:
        w = csv.writer(out)
        w.writerow(["id","has_TM","has_SP"])
        while True:
            header = f.readline().strip()
            if not header:
                break
            _seq = f.readline()
            labels = f.readline().strip()
            pid = header[1:]
            has_tm = any(c in labels for c in "HhBb")
            has_sp = "S" in labels
            total += 1
            if has_tm or has_sp:
                discarded += 1
            else:
                kept += 1
                kept_ids.append(pid)
                w.writerow([pid, 0, 0])
    return total, discarded, kept, kept_ids

# ---------------------------------------------------------------------------
# Main pipeline
# ---------------------------------------------------------------------------

def main():
    if len(sys.argv) < 2:
        sys.exit(__doc__)
    inp_fasta = "/content/PROTGPT_sequences_conserved.fasta"
    out_root  = "results"
    os.makedirs(out_root, exist_ok=True)

    # 1) Carga y filtra
    seqs = load_fasta(inp_fasta)
    print(f"[INFO] Secuencias totales: {len(seqs)}")
    seqs2 = quality_filter(seqs)
    print(f"[INFO] Tras filtros calidad: {len(seqs2)} "
          f"(descartadas {len(seqs)-len(seqs2)})")

    # 2) FASTA temporal filtrado
    with tempfile.TemporaryDirectory() as td:
        filt_fasta = os.path.join(td, "filtered.fasta")
        with open(filt_fasta, "w") as fh:
            for sid, seq in seqs2.items():
                fh.write(f">{sid}\n{seq}\n")

        # 3) Ejecuta TMbed
        pred_file = os.path.join(out_root, "tmbed.pred")
        print("[INFO] Ejecutando TMbed...")
        run_tmbed(filt_fasta, pred_file)

    # 4) Filtra predicciones y resume
    summary_csv = os.path.join(out_root, "filtered_summary.csv")
    total, discarded, kept, kept_ids = summarize_and_filter(pred_file, summary_csv)

    # 5) Genera nuevo FASTA con secuencias conservadas
    kept_fasta = os.path.join(out_root, "kept_sequences.fasta")
    with open(kept_fasta, "w") as fh:
        for pid in kept_ids:
            fh.write(f">{pid}\n{seqs2[pid]}\n")

    # 6) Informar al usuario
    print(f"[OK] Todos los resultados en '{out_root}':")
    print(f"  - Predicción TMbed   : {pred_file}")
    print(f"  - Resumen CSV        : {summary_csv}")
    print(f"  - FASTA conservadas  : {kept_fasta}")
    print(f"  - Total procesadas   : {total}")
    print(f"  - Descartadas (TM/SP): {discarded}")
    print(f"  - Conservar          : {kept}")

if __name__ == "__main__":
    main()



[INFO] Secuencias totales: 2702
[INFO] Tras filtros calidad: 1395 (descartadas 1307)
[INFO] Ejecutando TMbed...
[OK] Todos los resultados en 'results':
  - Predicción TMbed   : results/tmbed.pred
  - Resumen CSV        : results/filtered_summary.csv
  - FASTA conservadas  : results/kept_sequences.fasta
  - Total procesadas   : 1395
  - Descartadas (TM/SP): 856
  - Conservar          : 539


## 3. Immunogenicidad


In [ ]:
#!pip install fair-esm
!pip install biopython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.6 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
from typing import Union, List, Optional
import torch, numpy as np
from esm import FastaBatchedDataset, pretrained
from tqdm import tqdm


def extract_embeddings_chunked(
    model_name: str,
    fasta_file: Union[str, Path],
    output_dir: Union[str, Path],
    tokens_per_batch: int = 3000,
    seq_length: int = 1022,
    repr_layer: int = 33,
    batches_per_file: int = 1000,   # ⬅️ guarda cada 1 000 batches
):
    fasta_file = Path(fasta_file)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # 1) Modelo
    model, alphabet = pretrained.load_model_and_alphabet(model_name)
    model.eval()
    if torch.cuda.is_available():
        model = model.cuda()

    # 2) DataLoader
    dataset  = FastaBatchedDataset.from_file(str(fasta_file))
    batches  = dataset.get_batch_indices(tokens_per_batch, extra_toks_per_seq=1)
    collate  = alphabet.get_batch_converter(seq_length)
    loader   = torch.utils.data.DataLoader(
        dataset,
        batch_sampler=batches,
        collate_fn=collate,
        num_workers=4,
        pin_memory=True,
    )

    # 3) Buffers para acumular hasta batches_per_file
    buf_ids, buf_embs = [], []
    chunk_idx = 0         # número de archivo guardado

    with torch.inference_mode():
        for bidx, (labels, strs, toks) in enumerate(tqdm(loader, desc="Batches")):
            if torch.cuda.is_available():
                toks = toks.cuda(non_blocking=True)

            out = model(toks, repr_layers=[repr_layer], return_contacts=False)
            rep = out["representations"][repr_layer]   # (B, L+1, 1280)

            # → emb medio por secuencia
            for i, label in enumerate(labels):
                entry_id = label.split()[0]
                L = min(seq_length, len(strs[i]))
                emb = rep[i, 1:L+1].mean(0).detach().cpu()   # 1 280-D

                buf_ids.append(entry_id)
                buf_embs.append(emb)

            # ¿Hemos llegado a batches_per_file?
            if (bidx + 1) % batches_per_file == 0:
                _save_chunk(buf_ids, buf_embs, output_dir, chunk_idx)
                chunk_idx += 1
                buf_ids, buf_embs = [], []        # vacía buffers
                torch.cuda.empty_cache()

        # Al terminar, guarda lo que quede (último trozo)
        if buf_ids:
            _save_chunk(buf_ids, buf_embs, output_dir, chunk_idx)


def _save_chunk(ids: List[str], embs: List[torch.Tensor], outdir: Path, idx: int):
    """Convierte a numpy y guarda chunk_{idx}.npz"""
    X = torch.stack(embs).numpy().astype("float32")   # (n,1280)
    np.savez_compressed(outdir / f"chunk_{idx:04d}.npz",
                        ids=np.array(ids),
                        X=X)
    print(f"🔹 Guardado chunk_{idx:04d}.npz  ({len(ids)} secuencias)")

import sys
model_name = 'esm2_t33_650M_UR50D'
fasta_file = "/content/kept_sequences.fasta"
output_dir = 'immunology_prediction'
extract_embeddings_chunked(model_name, fasta_file, output_dir)
import os
import numpy as np

# Directorios y nombres
emb_dir = "/content/immunology_prediction"   # donde están chunk_000*.npz
out_dir = "/content/"
os.makedirs(out_dir, exist_ok=True)

final_name = "protein_embeddings"      # fichero final: unlabeled_ESM_ALL.npz

# Buffers
all_ids, all_X, chunk_index = [], [], []

# Recorre todos los .npz ordenados
npz_files = sorted(f for f in os.listdir(emb_dir) if f.endswith(".npz"))

for idx, fname in enumerate(npz_files, 1):
    path = os.path.join(emb_dir, fname)
    print(f"Cargando {idx}/{len(npz_files)} → {fname}")

    data = np.load(path, allow_pickle=False)
    X_chunk  = data["X"]                 # (m,1280)  float32
    ids_chunk = data["ids"]              # (m,)      str

    all_X.append(X_chunk)
    all_ids.append(ids_chunk)
    chunk_index.append(np.full(len(ids_chunk), idx-1, dtype=np.int16))  # opcional

# Concatenar todo
X_final   = np.vstack(all_X)             # (N,1280)
ids_final = np.concatenate(all_ids)      # (N,)

# Guardar
out_path = os.path.join(out_dir, f"{final_name}.npz")
np.savez_compressed(out_path,
                    X=X_final,
                    ids=ids_final,
                    chunks=np.concatenate(chunk_index))   # 'chunks' es opcional

print(f"\n✔️  Guardado archivo único: {out_path}")
print("   Embeddings totales:", X_final.shape[0])
import pandas as pd
from tensorflow.keras.models import load_model
from Bio import SeqIO  # pip install biopython
output_fasta="/content/immuno_sequences.fasta"
# 1) Carga el modelo y predice
modelo_immun = load_model('immupig_epoch500.keras')
probs = modelo_immun.predict(X_final).ravel()   # shape (n,)
input_fasta="/content/kept_sequences.fasta"
#
# Asumo que tienes en paralelo:
# ids_final: lista o array de identificadores (strings) para cada fila de X_final

# 2) DataFrame y CSV
df = pd.DataFrame({
    'id':    ids_final,
    'score': probs
})
df.to_csv('scores_immupig.csv', index=False)
print("CSV escrito: scores_immupig.csv")

# 3) Filtrar FASTA según umbral
threshold = 0.5
# crear un set con los ids que cumplen score > 0.5
ids_bajos = set(df.loc[df['score'] > threshold, 'id'])

total = 0
kept  = 0

with open(output_fasta, 'w') as out_f:
    for rec in SeqIO.parse(input_fasta, 'fasta'):
        total += 1
        if rec.id in ids_bajos:
            SeqIO.write(rec, out_f, 'fasta')
            kept += 1

discarded = total - kept

print(f"Secuencias originales: {total}")
print(f"Secuencias mantenidas (score > {threshold}): {kept}")
print(f"Secuencias descartadas: {discarded}")
print(f"FASTAs filtrados escritos en: {output_fasta}")



Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt


KeyboardInterrupt: 

In [ ]:
from pathlib import Path
from typing import Union, List, Optional
import torch, numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from Bio import SeqIO  # pip install biopython
input_fasta = "/content/kept_sequences.fasta"
output_dir = 'immunology_prediction'
X_final=np.load("/content/protein_embeddings.npz")['X']
ids_final=np.load("/content/protein_embeddings.npz")['ids']
output_fasta="/content/immuno_sequences.fasta"
modelo_immun = load_model('immupig_epoch500.keras')
probs = modelo_immun.predict(X_final).ravel()   # shape (n,)

#
# Asumo que tienes en paralelo:
# ids_final: lista o array de identificadores (strings) para cada fila de X_final

# 2) DataFrame y CSV
df = pd.DataFrame({
    'id':    ids_final,
    'score': probs
})
df.to_csv('scores_immupig.csv', index=False)
print("CSV escrito: scores_immupig.csv")

# 3) Filtrar FASTA según umbral
threshold = 0.5
# crear un set con los ids que cumplen score > 0.5
ids_bajos = set(df.loc[df['score'] > threshold, 'id'])

total = 0
kept  = 0

with open(output_fasta, 'w') as out_f:
    for rec in SeqIO.parse(input_fasta, 'fasta'):
        total += 1
        if rec.id in ids_bajos:
            SeqIO.write(rec, out_f, 'fasta')
            kept += 1

discarded = total - kept

print(f"Secuencias originales: {total}")
print(f"Secuencias mantenidas (score > {threshold}): {kept}")
print(f"Secuencias descartadas: {discarded}")
print(f"FASTAs filtrados escritos en: {output_fasta}")



17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
CSV escrito: scores_immupig.csv
Secuencias originales: 539
Secuencias mantenidas (score > 0.5): 288
Secuencias descartadas: 251
FASTAs filtrados escritos en: /content/immuno_sequences.fasta


In [ ]:
# 3) Filtrar FASTA según umbral
threshold = 0.5
# crear un set con los ids que cumplen score > 0.5
ids_bajos = set(df.loc[df['score'] > threshold, 'id'])

total = 0
kept  = 0

with open(output_fasta, 'w') as out_f:
    for rec in SeqIO.parse(input_fasta, 'fasta'):
        total += 1
        if rec.id in ids_bajos:
            SeqIO.write(rec, out_f, 'fasta')
            kept += 1

discarded = total - kept

print(f"Secuencias originales: {total}")
print(f"Secuencias mantenidas (score > {threshold}): {kept}")
print(f"Secuencias descartadas: {discarded}")
print(f"FASTAs filtrados escritos en: {output_fasta}")

Secuencias originales: 539
Secuencias mantenidas (score > 0.5): 288
Secuencias descartadas: 251
FASTAs filtrados escritos en: /content/immuno_sequences.fasta


#FILTRADO PLAUSIBILIDAD ESTRUCTURAL

## 1. Designable vs Undesignable folds




In [ ]:
%%time
#@title install
#@markdown install ESMFold, OpenFold and download Params (~2min 30s)
version = "1" # @param ["0", "1"]
model_name = "esmfold_v0.model" if version == "0" else "esmfold.model"
import os, time
if not os.path.isfile(model_name):
  # download esmfold params
  os.system("apt-get install aria2 -qq")
  os.system(f"aria2c -q -x 16 https://colabfold.steineggerlab.workers.dev/esm/{model_name} &")

  if not os.path.isfile("finished_install"):
    # install libs
    print("installing libs...")
    os.system("pip install -q omegaconf pytorch_lightning biopython ml_collections einops py3Dmol modelcif")
    os.system("pip install -q git+https://github.com/NVIDIA/dllogger.git")

    print("installing openfold...")
    # install openfold
    os.system(f"pip install -q git+https://github.com/sokrypton/openfold.git")

    print("installing esmfold...")
    # install esmfold
    os.system(f"pip install -q git+https://github.com/sokrypton/esm.git")
    os.system("touch finished_install")

  # wait for Params to finish downloading...
  while not os.path.isfile(model_name):
    time.sleep(5)
  if os.path.isfile(f"{model_name}.aria2"):
    print("downloading params...")
  while os.path.isfile(f"{model_name}.aria2"):
    time.sleep(5)

installing libs...
installing openfold...
installing esmfold...
CPU times: user 616 ms, sys: 61.7 ms, total: 678 ms
Wall time: 3min 47s
